In [4]:
from time import time

import random
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

train_data = pd.read_csv('./input/train.csv')
test_labels = pd.read_csv('./input/test.csv')
test_data = test_labels.drop('Sentiment', axis=1)
#train_data = df[:500]
#test_labels = test_labels[:100]
#test_data = test_data[:100]
print train_data.shape, test_labels.shape, test_data.shape

(18000, 5) (2000, 5) (2000, 4)


In [5]:
#Preprocessing
import re
from nltk.corpus import stopwords

def preprocess(row):
    phrase = row["Phrase"]
    phrase = re.sub("[0-9-]", "", phrase) #remove numbers
    phrase = phrase.lower() #lowercase
    phraseArray = phrase.split()
    #remove stopwords
    stops = set(stopwords.words("english"))
    nonstopwords = [w for w in phraseArray if not w in stops]
    return " ".join(nonstopwords) #return the preprocessed version

In [6]:
train_data["Preprocessed"] = train_data.apply(preprocess, axis=1)
#print(df)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word", \
                            tokenizer = None, \
                            preprocessor = None, \
                            stop_words = None, \
                            max_features = 5000)
t2 = time()
train_data_features = vectorizer.fit_transform(train_data['Preprocessed'])

train_data_features = train_data_features.toarray()
print "Cleaning time:", round(time()-t2, 3), "s"
#print(train_data_features)

#Vocabulary
#print(vectorizer.get_feature_names())

Cleaning time: 0.362 s


In [8]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
#Initialize with 100 trees
forest = RandomForestClassifier(n_estimators = 100)
t0 = time()
forest = forest.fit(train_data_features, train_data["Sentiment"])
print "Training time:", round(time()-t0, 3), "s"
train_result = forest.predict(train_data_features)
print "Accuracy score:", accuracy_score(train_result, train_data["Sentiment"].as_matrix())

Training time: 1211.104 s
Accuracy score: 0.920166666667


In [9]:
#Test
test_data["Preprocessed"] = test_data.apply(preprocess, axis=1)
test_data_features = vectorizer.transform(test_data["Preprocessed"])
test_data_features = test_data_features.toarray()

t1 = time()
result = forest.predict(test_data_features)
print "Prediction time:", round(time()-t1, 3), "s"
print "Accuracy score:", accuracy_score(result, test_labels["Sentiment"].as_matrix())

Prediction time: 1.332 s
Accuracy score: 0.557


In [20]:
accuracy_labels = [0,0,0,0]
correct_labels = test_labels["Sentiment"].as_matrix()
for i in range(0, len(result)):
    if(result[i] == correct_labels[i]):
        accuracy_labels[result[i] - 1] += 1
print accuracy_labels

[90, 796, 161, 67]
